# News Category Classification using Transformers

## 1. Load & Inspect the Dataset

### 1.1 Load the Dataset

In [1]:
import pandas as pd


DATA_PATH = "/kaggle/input/news-category-dataset/News_Category_Dataset_v3.json"

df = pd.read_json(DATA_PATH, lines=True)

print("Dataset shape:", df.shape)
df.head()

Dataset shape: (209527, 6)


,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


### 1.2 Remove Invalid Rows

In [4]:
df = df.dropna(subset=["category"])

df["headline"] = df["headline"].fillna("")
df["short_description"] = df["short_description"].fillna("")

print("Cleaned dataset shape:", df.shape)

Cleaned dataset shape: (209527, 6)


### 1.3 Create Final Text Column and Sanity Check

In [5]:
df["text"] = df["headline"].str.strip() + " " + df["short_description"].str.strip()

df = df[df["text"].str.strip() != ""]

df[["category", "text"]].head()


print("Number of categories:", df["category"].nunique())

df["category"].unique()[:10]

df["text"].str.len().describe()

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
Number of categories: 42


count    209522.000000
mean        173.628144
std          78.549381
min           4.000000
25%         123.000000
50%         171.000000
75%         208.000000
max        1487.000000
Name: text, dtype: float64